# 反爬：代理伺服器/IP

* 了解「IP 黑/白名單」的反爬蟲機制
* 「IP 黑/白名單」反爬蟲的因應策略

## 作業目標

* 目前程式中的 proxy_ips 是手動輸入的，請根據 https://free-proxy-list.net/ 寫一個可自動化抓取可用 Proxy 的 proxy_ips。




In [1]:
from bs4 import BeautifulSoup
import requests
import random

In [2]:
proxy_ips = []

# 對 https://free-proxy-list.net/ 發送請求，並從表格中整理出免費的代理伺服器

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Cafari/537.36'}
url = 'https://free-proxy-list.net/'
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'lxml')

for tr in soup.find('tbody').find_all('tr')[:100]:
    proxy = tr.find_all('td')[0].text + ':' + tr.find_all('td')[1].text
    proxy_ips.append(proxy)
    
print(proxy_ips)

['109.75.47.248:53422', '195.9.188.78:53281', '170.82.231.26:56805', '118.175.240.205:49799', '198.50.152.64:23500', '187.111.160.8:42579', '194.181.82.190:35447', '36.66.98.6:53281', '216.218.15.48:34377', '193.178.50.49:3128', '41.75.123.49:41263', '159.224.37.181:35306', '188.120.232.181:8118', '118.175.207.205:61293', '27.116.51.115:8080', '202.166.220.150:32324', '80.76.40.227:3128', '200.25.254.135:56897', '140.227.175.225:1000', '103.113.69.17:33694', '155.93.108.170:34997', '62.205.169.74:53281', '109.232.106.236:35423', '195.230.115.115:8080', '118.174.232.106:50491', '175.41.44.36:52581', '62.118.131.240:3128', '175.106.17.62:33337', '160.202.40.20:55655', '169.239.45.37:8080', '103.12.161.196:50514', '183.89.157.48:8213', '109.248.62.207:43558', '131.161.68.29:33716', '5.135.108.117:3128', '41.65.146.38:8080', '191.103.219.225:48612', '108.61.220.212:8118', '200.206.50.66:42515', '103.215.177.156:8888', '45.64.99.25:8080', '180.254.30.216:8080', '98.172.141.30:8080', '163.44

In [3]:
# 將免費的代理伺服器，發送至 http://ip.filefab.com/index.php 找出可用的代理伺服器

available_proxies = []

for i in range(100):
    ip = proxy_ips[i]
    print('Use', ip)
    try:
        resp = requests.get('http://ip.filefab.com/index.php',
                        proxies={'http': ip, 'https': ip}, timeout=10)
        soup = BeautifulSoup(resp.text, 'html5lib')
        print(soup.find('h1', id='ipd').text.strip())
        available_proxies.append(ip)
        if len(available_proxies) > 5:
            break
    except:
        print('Fail')
    
print(available_proxies)

Use 109.75.47.248:53422
Fail
Use 195.9.188.78:53281
Fail
Use 170.82.231.26:56805
Fail
Use 118.175.240.205:49799
Fail
Use 198.50.152.64:23500
Fail
Use 187.111.160.8:42579
Fail
Use 194.181.82.190:35447
Fail
Use 36.66.98.6:53281
Fail
Use 216.218.15.48:34377
Fail
Use 193.178.50.49:3128
Fail
Use 41.75.123.49:41263
Fail
Use 159.224.37.181:35306
Fail
Use 188.120.232.181:8118
Fail
Use 118.175.207.205:61293
Fail
Use 27.116.51.115:8080
Fail
Use 202.166.220.150:32324
Fail
Use 80.76.40.227:3128
Fail
Use 200.25.254.135:56897
Fail
Use 140.227.175.225:1000
Fail
Use 103.113.69.17:33694
Fail
Use 155.93.108.170:34997
Fail
Use 62.205.169.74:53281
Fail
Use 109.232.106.236:35423
Fail
Use 195.230.115.115:8080
Fail
Use 118.174.232.106:50491
Fail
Use 175.41.44.36:52581
Fail
Use 62.118.131.240:3128
Fail
Use 175.106.17.62:33337
Fail
Use 160.202.40.20:55655
Fail
Use 169.239.45.37:8080
Fail
Use 103.12.161.196:50514
Fail
Use 183.89.157.48:8213
Fail
Use 109.248.62.207:43558
Fail
Use 131.161.68.29:33716
Fail
Use 5.1

In [4]:
# 將前面找出的可用代理伺服器，再發送至 https://httpbin.org/ip 查看是否代理（偷換 IP）成功

for ip in available_proxies:
    print('Use', ip)
    try:
        resp = requests.get('https://httpbin.org/ip',
                        proxies={'http': ip, 'https': ip}, timeout=10)
        print(resp.json())
    except:
        print('Fail')